# This is reconstruction of SCADA_OTALA intrusion detector

In [66]:
# coding=utf-8
from scapy.all import *
import binascii
import struct
import time
import crcmod
import libnum

from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether

import numpy as np

A = []
Symbols = []

### SCADA_OTALA  Amit et,al. Training algorithm

In [67]:

def BuildSymbols(A):
    if A not in Symbols:
        #map state to alphabets
        Symbols.append(A)

def OTALA_train(A):
    print('Start SCADA_OTALA process at ', time.time())
    S = []
    T = []
    R = []
    Delta = [] #Time constrain [0,inf]
    Sigma = BuildSymbols(A) #Set of symbols [A11, Aij], i=register number, j=time
    c = 0
    try:
        print('All state length = ',len(A))
        for i in range(len(A)):
            start = time.time()
            c=i
            #print((A[i] == A[i+1]).all())
            while (A[i] == A[i+1]).all(): #if Registers at time i = Registers at time i+1
                print(i+1)
                if i+2 == len(A): #for manage index out of range
                    break
                i += 1
                
            if i+2 == len(A): #for manage index out of range
                break
            
            if A[i] not in S:
                S.append(A[i])
            if A[i+1] not in S:
                S.sppend(A[i+1])

            D = A[i]-A[i+1] #Added trasition of states
            t = [A[i],A[i+1],D,Delta]
            r = i-c
            if t not in T:
                T.append(t)
                Rt = [r]
                R.append(Rt)
            else:
                Rt.append(r)
            stop = time.time() - start
            print('Loop number: '+ i)
            print('Execution time: '+ stop)

        print('Create all Rt at ', time.time())
                
        for Rt in R:
            mean = np.average(Rt)
            stddev = np.std(Rt)
            upperBound = mean + 3*(stddev)
            lowerBound = mean + 3*(stddev)
            Delta = [lowerBound,upperBound]
            #updateTimeConstraint(t in T,Delta)
            
        print('Completed at ', time.time())
    except Exception as sniff_dec_err:
        traceback.print_exc()
        print(str(sniff_dec_err))
            
    return S,Sigma,T,c

In [68]:
%%latex
$$c = \sqrt{a^2 + b^2}$$

<IPython.core.display.Latex object>

# This is the Training session

In [69]:
def packet_analysis(pkt):
    #Add Registers values to Sets
    if TCP in pkt:
        if len(pkt[TCP].payload) > 6:
            #Check MODBUS
            #print(pkt[TCP].payload)
            payload_str = bytes(pkt[TCP].payload)[4:]
            #print(payload_str)
            mod_len = int.from_bytes(payload_str[:2], "big")
            mod_uid = int.from_bytes(payload_str[2:3], "big")
            mod_fc = int.from_bytes(payload_str[3:4], "big")
            mod_len = int.from_bytes(payload_str[4:5], "big")
            #print("Length = ",mod_len)
            #print("Function Codes = ",mod_fc)
            if mod_len > 0:
                #print(payload_str[5:])
                reg_state = []
                for i in range(0,mod_len-1,2):
                    value = int.from_bytes(payload_str[i+5:i+7], "big")
                    reg_state.append(value)
                A.append(np.asarray(reg_state))
                    
#sniffer
try:
    sniff(offline='DataSet/captures1_v2/captures1_v2/clean/eth2dump-clean-1h_1.pcap', prn=packet_analysis, store=0)
    S,Sigma,T,c = OTALA_train(A)
    print(c)
    print('Completed')

except Exception as sniff_dec_err:
    traceback.print_exc()
    print(str(sniff_dec_err))


Start SCADA_OTALA process at  1617866895.0637915
All state length =  11510
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
2

1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107


5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647


7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7661
7662
7663
7664
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7687
7688
7689
7690
7691
7692
7693
7694
7695
7696
7697
7698
7699
7700
7701
7702
7703
7704
7705
7706
7707
7708
7709
7710
7711
7712
7713
7714
7715
7716
7717
7718
7719
7720
7721
7722
7723
7724
7725
7726
7727
7728
7729
7730
7731
7732
7733
7734
7735
7736
7737
7738
7739
7740
7741
7742
7743
7744
7745
7746
7747
7748
7749
7750
7751
7752
7753
7754
7755
7756
7757
7758
7759
7760
7761
7762
7763
7764
7765
7766
7767
7768
7769
7770
7771
7772
7773
7774
7775
7776
7777
7778
7779
7780
7781
7782
7783
7784
7785
7786
7787
7788
7789
7790
7791
7792
7793
7794
7795
7796
7797
7798
7799
7800
7801
7802
7803
7804
7805
7806
7807
7808
7809
7810
7811
7812
7813
7814
7815
7816
7817
7818
7819
7820
7821
7822
7823
7824
7825
7826
7827
7828
7829
7830
7831
7832


9451
9452
9453
9454
9455
9456
9457
9458
9459
9460
9461
9462
9463
9464
9465
9466
9467
9468
9469
9470
9471
9472
9473
9474
9475
9476
9477
9478
9479
9480
9481
9482
9483
9484
9485
9486
9487
9488
9489
9490
9491
9492
9493
9494
9495
9496
9497
9498
9499
9500
9501
9502
9503
9504
9505
9506
9507
9508
9509
9510
9511
9512
9513
9514
9515
9516
9517
9518
9519
9520
9521
9522
9523
9524
9525
9526
9527
9528
9529
9530
9531
9532
9533
9534
9535
9536
9537
9538
9539
9540
9541
9542
9543
9544
9545
9546
9547
9548
9549
9550
9551
9552
9553
9554
9555
9556
9557
9558
9559
9560
9561
9562
9563
9564
9565
9566
9567
9568
9569
9570
9571
9572
9573
9574
9575
9576
9577
9578
9579
9580
9581
9582
9583
9584
9585
9586
9587
9588
9589
9590
9591
9592
9593
9594
9595
9596
9597
9598
9599
9600
9601
9602
9603
9604
9605
9606
9607
9608
9609
9610
9611
9612
9613
9614
9615
9616
9617
9618
9619
9620
9621
9622
9623
9624
9625
9626
9627
9628
9629
9630
9631
9632
9633
9634
9635
9636
9637
9638
9639
9640
9641
9642
9643
9644
9645
9646
9647
9648
9649
9650


11243
11244
11245
11246
11247
11248
11249
11250
11251
11252
11253
11254
11255
11256
11257
11258
11259
11260
11261
11262
11263
11264
11265
11266
11267
11268
11269
11270
11271
11272
11273
11274
11275
11276
11277
11278
11279
11280
11281
11282
11283
11284
11285
11286
11287
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
0
Completed


Traceback (most recent call last):
  File "<ipython-input-67-e8ea80b0033c>", line 31, in OTALA_train
    if A[i+1] not in S:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


### SCADA_OTALA Amit et,al. Detection algorithm

In [78]:
def findTransition(Ai,Ai1,D,T): #function is not definded
    for t in T:
        if t[0] == Ai and t[1]==Ai1 and t[2]==D:
            return t


def OTALA_Detect(A,S,Sigma,T,c):
    for i in range(len(A)):
        c=i
        
        while (A[i] == A[i+1]).all():
            print('State A'+str(i),A[i])
            print('State A'+str(i)+'+1',A[i+1])
            if i+2 == len(A): #for manage index out of range
                break
            i += 1
        if i+2 == len(A): #for manage index out of range
            break
        for symb in S:
            if (symb == A[i]).all() and (symb == A[i+1]).all():
                print(A[i])
                print(A[i+1])
                continue
            else:
                return "Alert"
        else:
            d = i-c
            D = A[i]-A[i+1]
            t = findTransition(A[i],A[i+1],D) #This function is ambigious
            if t not in T: #This function is ambigious t is not array of same size with T members.
                return "Alert"
        d=i-c
        Delta = t.Delta
        if d < Delta.lowerBound or d > Delta.upperBound:
            return "Alert"

# This is detection sessions

In [79]:
def packet_analysis(pkt):
    #Add Registers values to Sets
    if TCP in pkt:
        if len(pkt[TCP].payload) > 6:
            #Check MODBUS
            #print(pkt[TCP].payload)
            payload_str = bytes(pkt[TCP].payload)[4:]
            #print(payload_str)
            mod_len = int.from_bytes(payload_str[:2], "big")
            mod_uid = int.from_bytes(payload_str[2:3], "big")
            mod_fc = int.from_bytes(payload_str[3:4], "big")
            mod_len = int.from_bytes(payload_str[4:5], "big")
            #print("Length = ",mod_len)
            #print("Function Codes = ",mod_fc)
            if mod_len > 0:
                #print(payload_str[5:])
                reg_state = []
                for i in range(0,mod_len-1,2):
                    value = int.from_bytes(payload_str[i+5:i+7], "big")
                    reg_state.append(value)
                A.append(np.asarray(reg_state))
                    
#sniffer
try:
    sniff(offline='DataSet/captures1_v2/captures1_v2/mitm/eth2dump-mitm-change-1m-1h_1.pcap', prn=packet_analysis, store=0)
    OTALA_Detect(A,S,Sigma,T,c)
    print(c)
    print('Completed')

except Exception as sniff_dec_err:
    traceback.print_exc()
    print(str(sniff_dec_err))

State A0 [ 1  0  0  0  0  0 30  0  0  0  0]
State A0+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A2 [ 1  0  0  0  0  0 30  0  0  0  0]
State A2+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A3 [ 1  0  0  0  0  0 30  0  0  0  0]
State A3+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A4 [ 1  0  0  0  0  0 30  0  0  0  0]
State A4+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A5 [ 1  0  0  0  0  0 30  0  0  0  0]
State A5+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A6 [ 1  0  0  0  0  0 30  0  0  0  0]
State A6+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A7 [ 1  0  0  0  0  0 30  0  0  0  0]
State A7+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A8 [ 1  0  0  0  0  0 30  0  0  0  0]
State A8+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1

State A382+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A383 [ 1  0  0  0  0  0 30  0  0  0  0]
State A383+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A384 [ 1  0  0  0  0  0 30  0  0  0  0]
State A384+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A385 [ 1  0  0  0  0  0 30  0  0  0  0]
State A385+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A386 [ 1  0  0  0  0  0 30  0  0  0  0]
State A386+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A387 [ 1  0  0  0  0  0 30  0  0  0  0]
State A387+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A388 [ 1  0  0  0  0  0 30  0  0  0  0]
State A388+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A389 [ 1  0  0  0  0  0 30  0  0  0  0]
State A389+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A390 [ 1  0  0  0  0  0 30  0  0  0  0]
State A390+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A391 [ 1  0  0  0  0  0 30  0  0  0  0]
State A391+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A392 [ 1  0  0  0  0  0 30  0  0  0  0]
State A392+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A393 [

State A721+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A722 [ 1  0  0  0  0  0 30  0  0  0  0]
State A722+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A723 [ 1  0  0  0  0  0 30  0  0  0  0]
State A723+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A724 [ 1  0  0  0  0  0 30  0  0  0  0]
State A724+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A725 [ 1  0  0  0  0  0 30  0  0  0  0]
State A725+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A726 [ 1  0  0  0  0  0 30  0  0  0  0]
State A726+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A727 [ 1  0  0  0  0  0 30  0  0  0  0]
State A727+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A728 [ 1  0  0  0  0  0 30  0  0  0  0]
State A728+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A729 [ 1  0  0  0  0  0 30  0  0  0  0]
State A729+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A730 [ 1  0  0  0  0  0 30  0  0  0  0]
State A730+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A731 [ 1  0  0  0  0  0 30  0  0  0  0]
State A731+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A732 [

State A1056+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1057 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1057+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1058 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1058+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1059 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1059+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1060 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1060+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1061 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1061+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1062 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1062+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1063 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1063+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1064 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1064+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1065 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1065+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1066 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1066+1 [ 1  0  0  0  0  0 30  0  

State A1330 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1330+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1331 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1331+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1332 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1332+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1333 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1333+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1334 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1334+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1335 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1335+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1336 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1336+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1337 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1337+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1338 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1338+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1339 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1339+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A1340 [ 1  0  0  0  0  0 30  0  0 

...

State A9911+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9912 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9912+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9913 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9913+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9914 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9914+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9915 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9915+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9916 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9916+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9917 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9917+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9918 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9918+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9919 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9919+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9920 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9920+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9921 [ 1  0  0  0  0  0 30  0  0  0  0]
State A9921+1 [ 1  0  0  0  0  0 30  0  

State A10279+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10280 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10280+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10281 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10281+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10282 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10282+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10283 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10283+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10284 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10284+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10285 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10285+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10286 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10286+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10287 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10287+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10288 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10288+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10289 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10289+1 [ 1  

State A10589 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10589+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10590 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10590+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10591 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10591+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10592 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10592+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10593 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10593+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10594 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10594+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10595 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10595+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10596 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10596+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10597 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10597+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10598 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10598+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10599 [ 1  0 

State A10874 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10874+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10875 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10875+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10876 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10876+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10877 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10877+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10878 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10878+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10879 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10879+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10880 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10880+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10881 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10881+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10882 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10882+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10883 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10883+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A10884 [ 1  0 

State A11211+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11212 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11212+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11213 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11213+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11214 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11214+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11215 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11215+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11216 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11216+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11217 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11217+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11218 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11218+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11219 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11219+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11220 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11220+1 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11221 [ 1  0  0  0  0  0 30  0  0  0  0]
State A11221+1 [ 1  